In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
from sklearn.preprocessing import StandardScaler
from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA
from sklearn import datasets, metrics

In [2]:
#load the dataset
heartdisease_df=pd.read_csv("heart.csv")
# Define the features and the outcome
X = heartdisease_df.iloc[:, :13]
y = heartdisease_df.iloc[:, 13]
# Replace missing values (marked by ?) with a 0
X = X.replace(to_replace='?', value=0)
# Binarize y so that 1 means heart disease diagnosis and 0 means no diagnosis
y = np.where(y > 0, 0, 1)
# Standarizing the features
scaler = StandardScaler()
X_std = scaler.fit_transform(X)


1.n_components = 2 olarak ayarlayarak kalp hastalığı verilerine GMM uygulayın. Çözümünüz için ARI ve siluet skorlarına bakın ve önceki derslerde k-means ve hiyerarşik kümeleme çözümleriyle karşılaştırın. Hangi algoritma daha iyi performans gösteriyor?

In [3]:
gmm_cluster = GaussianMixture(n_components=2, random_state=123)
clusters = gmm_cluster.fit_predict(X_std)

In [4]:
print("GMM çözümünün Düzeltilmiş Rand Endeksi (ARI): {:.3f}"
      .format(metrics.adjusted_rand_score(y, clusters)))
print("GMM çözümünün siluet skoru                  : {:.3f}"
      .format(metrics.silhouette_score(X_std, clusters, metric='euclidean')))

GMM çözümünün Düzeltilmiş Rand Endeksi (ARI): 0.190
GMM çözümünün siluet skoru                  : 0.135


k-ortalamanın degerler:  
siluet degeri= 0.16759734983094837 n= 2 icin ayarlanmıs rand indeksi= 0.3951165387976022  
hiyerarşik kümelemenin farklı metodları için degerler:  
average  icin Yığınsal Kümeleme Sonuçlarının Ayarlanmış Rand Endeksi: -0.0010800413233203012  
average  icin The silhoutte score of the Agglomerative Clustering solution: 0.3532620013624347  
complete  icin Yığınsal Kümeleme Sonuçlarının Ayarlanmış Rand Endeksi: -0.0010800413233203012  
complete  icin The silhoutte score of the Agglomerative Clustering solution: 0.3532620013624347  
ward  icin Yığınsal Kümeleme Sonuçlarının Ayarlanmış Rand Endeksi: 0.2454680392170414  
ward  icin The silhoutte score of the Agglomerative Clustering solution: 0.1325316617259095  

tüm değerleri inceleyince ARI açısından en iyisi:k-ortalama siluet açısından en iyisi:hiyerarsik kümelemenin average ve complete metodları oldukları görülüyor.GMM bu dataset için en geride kalıyor bu metriklere bakınca.


**2.** Scikit-learn ile GMM uygulanırken kullanılan parametrelerden biri de covariance_type'dir. Bu parametre, kullanılacak kovaryans parametrelerinin türünü belirler. Özellikle, belirtebileceğiniz dört tür vardır:

full: Bu varsayılandır. Her bileşenin kendi genel kovaryans matrisi vardır.  
tied: Tüm bileşenler aynı genel kovaryans matrisini paylaşır.  
diag: Her bir bileşenin kendi çapraz kovaryans matrisi vardır.  
spherical: Her bir bileşenin kendine ait tek bir varyansı vardır.    

Bunları deneyin. Hangisi ARI ve siluet skorları açısından daha iyi performans gösteriyor?

In [5]:
cov_type=["full","tied","diag","spherical"]
for i in cov_type:
    gmm_cluster = GaussianMixture(n_components=2,covariance_type=i, random_state=123)
    clusters = gmm_cluster.fit_predict(X_std)
    print(i," kovaryans tipi icin GMM çözümünün Düzeltilmiş Rand Endeksi (ARI): {:.3f}"
      .format(metrics.adjusted_rand_score(y, clusters)))
    print(i," kovaryans tipi icin GMM çözümünün siluet skoru                  : {:.3f}"
      .format(metrics.silhouette_score(X_std, clusters, metric='euclidean')))

full  kovaryans tipi icin GMM çözümünün Düzeltilmiş Rand Endeksi (ARI): 0.190
full  kovaryans tipi icin GMM çözümünün siluet skoru                  : 0.135
tied  kovaryans tipi icin GMM çözümünün Düzeltilmiş Rand Endeksi (ARI): 0.190
tied  kovaryans tipi icin GMM çözümünün siluet skoru                  : 0.135
diag  kovaryans tipi icin GMM çözümünün Düzeltilmiş Rand Endeksi (ARI): 0.190
diag  kovaryans tipi icin GMM çözümünün siluet skoru                  : 0.135
spherical  kovaryans tipi icin GMM çözümünün Düzeltilmiş Rand Endeksi (ARI): 0.178
spherical  kovaryans tipi icin GMM çözümünün siluet skoru                  : 0.107


4 ayrı kovaryans tipi içinde **spherical** iki metrik açısından da en kötüsü olmak üzere diğer üçünün değerleri aynı çıktı.